# MultiEval Example

This notebook demonstrates a basic parameter sweep with LensKits `MultiEval` class.

## Setup

We first need to import our libraries.

In [17]:
import sys
sys.path

#import os
#os.getcwd()
#os.chdir('C:\\Users\\Jacob\\Documents\\GitHub\\lenskit_confidence\\lenskit')

['C:\\Users\\Jacob\\Documents\\GitHub\\lenskit_confidence',
 'C:\\Users\\Jacob\\Documents\\GitHub\\lenskit_confidence',
 'C:\\Users\\Jacob\\Documents\\GitHub\\lenskit_confidence\\examples',
 'C:\\Users\\Jacob\\anaconda3\\python38.zip',
 'C:\\Users\\Jacob\\anaconda3\\DLLs',
 'C:\\Users\\Jacob\\anaconda3\\lib',
 'C:\\Users\\Jacob\\anaconda3',
 '',
 'C:\\Users\\Jacob\\anaconda3\\lib\\site-packages',
 'C:\\Users\\Jacob\\anaconda3\\lib\\site-packages\\binpickle-0.3.2-py3.8.egg',
 'C:\\Users\\Jacob\\anaconda3\\lib\\site-packages\\pyarrow-2.0.0-py3.8-win-amd64.egg',
 'C:\\Users\\Jacob\\anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\Jacob\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Jacob\\anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\Jacob\\anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Jacob\\.ipython']

In [18]:
sys.path.insert(0,'C:\\Users\\Jacob\\Documents\\GitHub\\lenskit_confidence')

In [ ]:
#set PYTHONPATH=C:\\Users\\Jacob\\Documents\\GitHub\\lenskit_confidence\\lenskit

In [19]:
from lenskit.batch import MultiEval
from lenskit.crossfold import partition_users, SampleN
from lenskit.algorithms import basic, als, item_knn, user_knn
from lenskit.datasets import MovieLens
from lenskit import topn, util, metrics
from lenskit.metrics import predict
import pandas as pd
import matplotlib.pyplot as plt

In [20]:
util.log_to_notebook()

[   INFO] lenskit.util.log log already initialized
[   INFO] lenskit.util.log log already initialized
[   INFO] lenskit.util.log notebook logging configured
[   INFO] lenskit.util.log notebook logging configured
[   INFO] lenskit.util.log notebook logging configured


In [16]:
# move elsewhere
from lenskit.algorithms.bias import Bias
from lenskit.metrics.predict import rmse
from lenskit import datasets
ratings = datasets.MovieLens('../data/ml-latest-small').ratings
bias = Bias()
print(ratings.head())
print(len(ratings))

bias.fit(ratings)

batch.predict(bias, ratings)

   user  item  rating   timestamp
0     1    31     2.5  1260759144
1     1  1029     3.0  1260759179
2     1  1061     3.0  1260759182
3     1  1129     2.0  1260759185
4     1  1172     4.0  1260759205
100004
[   INFO] lenskit.algorithms.bias building bias model for 100004 ratings
[   INFO] lenskit.algorithms.bias building bias model for 100004 ratings
[   INFO] lenskit.algorithms.bias global mean: 3.544
[   INFO] lenskit.algorithms.bias global mean: 3.544
[   INFO] numexpr.utils NumExpr defaulting to 4 threads.
[   INFO] numexpr.utils NumExpr defaulting to 4 threads.
[   INFO] lenskit.algorithms.bias computed means for 9066 items
[   INFO] lenskit.algorithms.bias computed means for 9066 items
[   INFO] lenskit.algorithms.bias computed means for 671 users
[   INFO] lenskit.algorithms.bias computed means for 671 users
[   INFO] lenskit.sharing.shm serialized Bias(ud=0.0, id=0.0) to 954 pickle bytes with 6 buffers of 233688 bytes
[   INFO] lenskit.sharing.shm serialized Bias(ud=0.0, id

,user,item,rating,timestamp,prediction
0,1,31,2.5,1260759144,2.166043
1,1,1029,3.0,1260759179,2.689852
2,1,1061,3.0,1260759182,2.532926
3,1,1129,2.0,1260759185,2.299971
4,1,1172,4.0,1260759205,3.248341
...,...,...,...,...,...
99999,671,6268,2.5,1065579370,3.075544
100000,671,6269,4.0,1065149201,4.218401
100001,671,6365,4.0,1070940363,3.343837
100002,671,6385,2.5,1070979663,3.780090


In [ ]:
import pandas as pd
from lenskit import batch, topn, datasets
from lenskit.metrics.predict import rmse
from lenskit.crossfold import partition_users, SampleN, partition_rows
from lenskit.algorithms import als, Recommender
from lenskit.algorithms import basic, als, item_knn, user_knn
# Load MovieLens data in expected format
ml = datasets.MovieLens('../data/ml-latest-small').ratings
#print(ml.head())

all_recs = []
all_preds = []
all_test = []

samp = SampleN(5)
#print(samp)
#pairs = list(partition_users(ml.ratings, 5, SampleN(5)))
nbhd_sizes = [5]

for f in nbhd_sizes:
    print('Starting neighborhood size: ', f)
    for train, test in partition_rows(ml, 2): #partition_users(ml, 5, samp):
        print(len(train))
        print(len(test))
        all_test.append(test)
        print('Configuring algorithm...')
        # configure the algorithm
        algo = user_knn.UserUser(nnbrs = f) #als.BiasedMF(50)
        # make sure it can recommend, not just predict
        algo = Recommender.adapt(algo)
        # train the algorithm
        algo.fit(train)
        print('Starting prediction process...')
        # predict ratings
        preds = batch.predict(algo, test)
        preds['nnbhrs'] = f
        all_preds.append(preds)
        print('Starting recommendation process...')
        # produce recommendations
        users = test['user'].unique()
        recs = batch.recommend(algo, users, 20)
        all_recs.append(recs)
        print('Finishing neighborhood size: ', f)

# integrate test data; since splits are disjoint users
# for a single data set, this is sufficient
recs = pd.concat(all_recs, ignore_index=True)
preds = pd.concat(all_preds, ignore_index=True)
test = pd.concat(all_test, ignore_index=True)
# compute RMSE
print('RMSE:', rmse(preds['prediction'],preds['rating']))

In [6]:
ml = datasets.MovieLens('../data/ml-latest-small').ratings
#ml = datasets.MovieLens('../data/ml-1m').ratings
#mlsmall = MovieLens('../data/ml-10m')
#mlsmall = MovieLens('../data/ml-20m')

In [21]:
import pandas as pd
from lenskit import batch, topn, datasets
from lenskit.metrics.predict import rmse
from lenskit.crossfold import partition_users, SampleN, partition_rows
from lenskit.algorithms import als, Recommender
from lenskit.algorithms import basic, als, item_knn, user_knn
# Load MovieLens data in expected format

#print(ml.head())

all_recs = []
all_preds = []
all_test = []

samp = SampleN(5)
#print(samp)
num_cv = 5
pairs = list(partition_users(ml, num_cv, SampleN(5)))
nbhd_sizes = [10, 15]
rec_list_size = 20

for f in nbhd_sizes:
    print('Starting neighborhood size: ', f)
    for p in range(len(pairs)): #partition_users(ml, 5, samp): #partition_rows(ml, 5): #partition_users(ml, 5, samp):
        #print(len(train))
        #print(len(test))
        
        train = pairs[p].train
        test = pairs[p].test
        
        all_test.append(test)
        print('Configuring algorithm...')
        # configure the algorithm
        algo = user_knn.UserUser(nnbrs = f) #als.BiasedMF(50)
        # make sure it can recommend, not just predict
        algo = Recommender.adapt(algo)
        # train the algorithm
        algo.fit(train)
        print('Starting prediction process...')
        # predict ratings
        preds = batch.predict(algo, test, n_jobs = 2)
        preds['nnbhrs'] = f
        preds['RunId'] = p + 1
        all_preds.append(preds)
        print('Starting recommendation process...')
        # produce recommendations
        users = test['user'].unique()
        recs = batch.recommend(algo, users, rec_list_size)
        recs['nnbhrs'] = f ####
        recs['RunId'] = p + 1
        all_recs.append(recs)
        print('Finishing neighborhood size: ', f)

# integrate test data; since splits are disjoint users
# for a single data set, this is sufficient
recs = pd.concat(all_recs, ignore_index=True)
preds = pd.concat(all_preds, ignore_index=True)
test = pd.concat(all_test, ignore_index=True)
# compute RMSE
#print('RMSE:', rmse(preds['prediction'],preds['rating']))

[   INFO] lenskit.crossfold partitioning 100004 rows for 671 users into 5 partitions
[   INFO] lenskit.crossfold partitioning 100004 rows for 671 users into 5 partitions
[   INFO] lenskit.crossfold partitioning 100004 rows for 671 users into 5 partitions
[   INFO] lenskit.crossfold fold 0: selecting test ratings
[   INFO] lenskit.crossfold fold 0: selecting test ratings
[   INFO] lenskit.crossfold fold 0: selecting test ratings
[   INFO] lenskit.crossfold fold 0: partitioning training data
[   INFO] lenskit.crossfold fold 0: partitioning training data
[   INFO] lenskit.crossfold fold 0: partitioning training data
[   INFO] lenskit.crossfold fold 1: selecting test ratings
[   INFO] lenskit.crossfold fold 1: selecting test ratings
[   INFO] lenskit.crossfold fold 1: selecting test ratings
[   INFO] lenskit.crossfold fold 1: partitioning training data
[   INFO] lenskit.crossfold fold 1: partitioning training data
[   INFO] lenskit.crossfold fold 1: partitioning training data
[   INFO] len

[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.batch._predict generating 670 predictions for 134 users (setup took  15ms)
[   INFO] lenskit.batch._predict generating 670 predictions for 134 users (setup took  16ms)
[   INFO] lenskit.batch._predict generating 670 predictions for 134 users (setup took  17ms)
[   INFO] lenskit.batch._predict generated 670 predictions for 134 users in 8.80s
[   INFO] lenskit.batch._predict generated 670 predictions for 134 users in 8.80s
[   INFO] lenskit.batch._predict generated 670 predictions for 134 users in 8.80s
Starting recommendation process...
[   INFO] lenskit.sharing.shm serialized TopN/UserUser(nnbrs=10, min_sim=0) to 1121 pickle bytes with 13 buffers of 2983896 bytes
[   INFO] lenskit.sharing.shm serialized TopN/UserUser(nnbrs=10, min_sim=0) to 112

[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.batch._predict generating 675 predictions for 135 users (setup took  17ms)
[   INFO] lenskit.batch._predict generating 675 predictions for 135 users (setup took  19ms)
[   INFO] lenskit.batch._predict generating 675 predictions for 135 users (setup took  21ms)
[   INFO] lenskit.batch._predict generated 675 predictions for 135 users in 8.81s
[   INFO] lenskit.batch._predict generated 675 predictions for 135 users in 8.81s
[   INFO] lenskit.batch._predict generated 675 predictions for 135 users in 8.81s
Starting recommendation process...
[   INFO] lenskit.sharing.shm serialized TopN/UserUser(nnbrs=15, min_sim=0) to 1121 pickle bytes with 13 buffers of 2983816 bytes
[   INFO] lenskit.sharing.shm serialized TopN/UserUser(nnbrs=15, min_sim=0) to 1121 pickle bytes with 13 buffers of 2983816 bytes
[   INFO] lenskit.sharing.sh

[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.batch._predict generating 670 predictions for 134 users (setup took  16ms)
[   INFO] lenskit.batch._predict generating 670 predictions for 134 users (setup took  17ms)
[   INFO] lenskit.batch._predict generating 670 predictions for 134 users (setup took  18ms)
[   INFO] lenskit.batch._predict generated 670 predictions for 134 users in 9.17s
[   INFO] lenskit.batch._predict generated 670 predictions for 134 users in 9.17s
[   INFO] lenskit.batch._predict generated 670 predictions for 134 users in 9.17s
Starting recommendation process...
[   INFO] lenskit.sharing.shm serialized TopN/UserUser(nnbrs=15, min_sim=0) to 1121 pickle bytes with 13 buffers of 2983976 bytes
[   INFO] lenskit.sharing.shm serialized TopN/UserUser(nnbrs=15, min_sim=0) to 1121 pickle bytes with 13 buffers of 2983976 bytes
[   INFO] lenskit.sharing.shm serialized TopN/UserUser(nnbrs=15, min_sim=0) to 1121 pickle bytes with 13

In [ ]:
len(pairs)

In [22]:
recs.head() # RECS

,item,score,user,rank,nnbhrs,RunId
0,107559,5.065356,1,1,10,1
1,106441,5.044886,1,2,10,1
2,73290,4.854950,1,3,10,1
3,3216,4.820168,1,4,10,1
4,4754,4.820168,1,5,10,1


In [23]:
preds.head()

,user,item,rating,timestamp,prediction,nnbhrs,RunId
0,1,1263,2.0,1260759151,2.954023,10,1
1,1,1371,2.5,1260759135,1.573504,10,1
2,1,1287,2.0,1260759187,2.660112,10,1
3,1,1953,4.0,1260759191,2.949838,10,1
4,1,2105,4.0,1260759139,2.733989,10,1


In [24]:
test.head() # TRUTH

,user,item,rating,timestamp
0,1,1263,2.0,1260759151
1,1,1371,2.5,1260759135
2,1,1287,2.0,1260759187
3,1,1953,4.0,1260759191
4,1,2105,4.0,1260759139


In [25]:
# compute nDCG for each user
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg)
rla.add_metric(topn.precision)
metrics = rla.compute(recs, test)


[   INFO] lenskit.topn analyzing 26840 recommendations (6710 truth rows)
[   INFO] lenskit.topn analyzing 26840 recommendations (6710 truth rows)
[   INFO] lenskit.topn analyzing 26840 recommendations (6710 truth rows)
[   INFO] lenskit.topn using rec key columns ['nnbhrs', 'RunId', 'user']
[   INFO] lenskit.topn using rec key columns ['nnbhrs', 'RunId', 'user']
[   INFO] lenskit.topn using rec key columns ['nnbhrs', 'RunId', 'user']
[   INFO] lenskit.topn using truth key columns ['user']
[   INFO] lenskit.topn using truth key columns ['user']
[   INFO] lenskit.topn using truth key columns ['user']
[   INFO] lenskit.topn collecting truth data
[   INFO] lenskit.topn collecting truth data
[   INFO] lenskit.topn collecting truth data
[   INFO] lenskit.topn collecting metric results
[   INFO] lenskit.topn collecting metric results
[   INFO] lenskit.topn collecting metric results
[   INFO] lenskit.util.parallel setting up in-process worker
[   INFO] lenskit.util.parallel setting up in-proce

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [26]:
# compute nDCG for each user
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg)
rla.add_metric(topn.precision)

metrics = []

for f in nbhd_sizes:
    rec_f = recs[recs['nnbhrs'] == f]
    metrics_f = rla.compute(rec_f, test)
    metrics.append(metrics_f)



[   INFO] lenskit.topn analyzing 13420 recommendations (6710 truth rows)
[   INFO] lenskit.topn analyzing 13420 recommendations (6710 truth rows)
[   INFO] lenskit.topn analyzing 13420 recommendations (6710 truth rows)
[   INFO] lenskit.topn using rec key columns ['nnbhrs', 'RunId', 'user']
[   INFO] lenskit.topn using rec key columns ['nnbhrs', 'RunId', 'user']
[   INFO] lenskit.topn using rec key columns ['nnbhrs', 'RunId', 'user']
[   INFO] lenskit.topn using truth key columns ['user']
[   INFO] lenskit.topn using truth key columns ['user']
[   INFO] lenskit.topn using truth key columns ['user']
[   INFO] lenskit.topn collecting truth data
[   INFO] lenskit.topn collecting truth data
[   INFO] lenskit.topn collecting truth data
[   INFO] lenskit.topn collecting metric results
[   INFO] lenskit.topn collecting metric results
[   INFO] lenskit.topn collecting metric results
[   INFO] lenskit.util.parallel setting up in-process worker
[   INFO] lenskit.util.parallel setting up in-proce

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
# and compute overall system performance
print('NDCG:', metrics.groupby('nnbhrs')['ndcg'].mean())
print('Precision:', metrics.groupby('nnbhrs')['precision'].mean())

In [ ]:
### FOR NEIGHBORHOOD-BASED METHODS ONLY ###
ndcg = raw_ndcg.join(runs[['AlgoClass', 'nnbrs']], on = 'RunId')
ndcg.head()

In [ ]:
import pandas as pd
from lenskit import batch, topn, datasets
from lenskit.algorithms.bias import Bias
from lenskit.metrics.predict import rmse
#from lenskit import datasets

ml = datasets.MovieLens('../data/ml-latest-small').ratings

ratings = ml
bias = Bias()
bias.fit(ratings[:-1000])

preds = batch.predict(bias, ratings[-1000:])
print(rmse(preds['prediction'], preds['rating']))
preds.head()

In [ ]:
preds['err'] = (preds['prediction'] - preds['rating'])**2

In [ ]:
preds

Progress bars are useful:

In [ ]:
from tqdm.notebook import tqdm_notebook as tqdm
tqdm.pandas()

It takes a little while to run things, and can get kinda quiet in here. Let's set up logging so we can see the logging output in the notebook's message stream:

In [ ]:
util.log_to_notebook()

Then set up the data access.

In [ ]:
mlsmall = MovieLens('../data/ml-latest-small')
#mlsmall = MovieLens('../data/ml-1m')
#mlsmall = MovieLens('../data/ml-10m')
#mlsmall = MovieLens('../data/ml-20m')


## Experiment

We're going to run our evaluation and store its output in the `my-eval` directory, generating 20-item recommendation lists::

In [ ]:
eval = MultiEval('my-eval', recommend = 0)

We're going to use a 5-fold cross-validation setup.  We save the data into a list in memory so we have access to the test data later.  In a larger experiment, you might write the partitions to disk and pass the file names to `add_datasets`.

In [ ]:
pairs = list(partition_users(mlsmall.ratings, 5, SampleN(5)))
eval.add_datasets(pairs, name = 'ML-Small')

In [ ]:
eval.add_algorithms([item_knn.ItemItem(nnbrs = f) for f in [10, 25, 50, 100]], 
                    attrs = ['nnbrs'], name = 'ItemKNN')

In [ ]:
eval.add_algorithms([user_knn.UserUser(nnbrs = f) for f in [10, 25, 50, 100]], 
                    attrs = ['nnbrs'], name = 'UserKNN')

And add a popular baseline for comparison:

In [ ]:
eval.add_algorithms(basic.Popular(), name = 'Pop')

And finally, we will run the experiment!

In [ ]:
eval.run(progress = tqdm)

In [ ]:
from lenskit.algorithms.bias import Bias
from lenskit.metrics.predict import rmse
from lenskit import datasets
ratings = mlsmall.ratings #datasets.MovieLens('data/ml-latest-small').ratings
bias = Bias()
bias.fit(ratings)
#<lenskit.algorithms.bias.Bias object at ...>
preds = predict(bias, ratings)
preds.head()

## Analysis

Now that the experiment is run, we can read its outputs.

First the run metadata:

In [ ]:
preds = predict(eval, mlsmall)

In [ ]:
runs = pd.read_csv('my-eval/runs.csv')
runs.set_index('RunId', inplace = True)
runs.head()

This describes each run - a data set, partition, and algorithm combination.  To evaluate, we need to get the actual recommendations, and combine them with this:

In [ ]:
recs = pd.read_parquet('my-eval/recommendations.parquet')
recs.head()

We're going to compute per-(run,user) evaluations of the recommendations *before* combining with metadata. 

In order to evaluate the recommendation list, we need to build a combined set of truth data. Since this is a disjoint partition of users over a single data set, we can just concatenate the individual test frames:

In [ ]:
truth = pd.concat((p.test for p in pairs), ignore_index = True)

Now we can set up an analysis and compute the results.

In [ ]:
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg) # precision, recall, recip_rank, dcg, ndcg
rla.add_metric(topn.precision)
#rla.add_metric(predict.rmse)
raw_ndcg = rla.compute(recs, truth)
raw_ndcg.head()

In [ ]:
raw_ndcg

Next, we need to combine this with our run data, so that we know what algorithms and configurations we are evaluating:

In [ ]:
### FOR NEIGHBORHOOD-BASED METHODS ONLY ###
ndcg = raw_ndcg.join(runs[['AlgoClass', 'nnbrs']], on = 'RunId')
ndcg.head()

We can compute the overall average performance for each algorithm configuration - fillna makes the group-by happy with Popular's lack of a feature count:

In [ ]:
### FOR NEIGHBORHOOD-BASED METHODS ONLY ###
ndcg.fillna(0).groupby(['AlgoClass', 'nnbrs'])['ndcg','precision'].mean()

Now, we can plot this:

In [ ]:
### FOR NEIGHBORHOOD-BASED METHODS ONLY ###
mf_scores = ndcg.groupby(['AlgoClass', 'nnbrs'])['ndcg'].mean().reset_index()
#pop_score = ndcg[ndcg['AlgoClass'] == 'Popular']['ndcg'].mean()
#plt.axhline(pop_score, color='grey', linestyle='--', label='Popular')
for algo, data in mf_scores.groupby('AlgoClass'):
    plt.plot(data['nnbrs'], data['ndcg'], label=algo)
plt.legend()
plt.xlabel('nnbrs')
plt.ylabel('nDCG')

In [ ]:
### FOR NEIGHBORHOOD-BASED METHODS ONLY ###
mf_scores = ndcg.groupby(['AlgoClass', 'nnbrs'])['precision'].mean().reset_index()
#pop_score = ndcg[ndcg['AlgoClass'] == 'Popular']['ndcg'].mean()
#plt.axhline(pop_score, color='grey', linestyle='--', label='Popular')
for algo, data in mf_scores.groupby('AlgoClass'):
    plt.plot(data['nnbrs'], data['precision'], label=algo)
plt.legend()
plt.xlabel('nnbrs')
plt.ylabel('Precision')